In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
import numpy as np

In [3]:
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white") 

# display plots in this notebook
%matplotlib inline

## use conv layer

In [4]:
tf.reset_default_graph()

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
Z_dim = 100
batch_size = 128

In [7]:
X = tf.placeholder(tf.float32, shape=[None, X_dim], name='X')

In [8]:
Z = tf.placeholder(tf.float32, shape=[None, Z_dim], name='Z')

In [9]:
def sample_Z(m, n):
    return np.random.uniform(-1, 1, size=[m, n])

In [10]:
def generator(z, reuse=False):
    with tf.variable_scope('G', reuse=reuse) as scope:
        G_dense = tf.layers.dense(z, units=14*14*200, activation=tf.nn.relu)
        G_reshape = tf.reshape(G_dense, [-1, 14, 14, 1])
        G_upsample = tf.image.resize_images(G_reshape, [28, 28])
        G_conv1 = tf.layers.conv2d(G_upsample, filters=64, kernel_size=[3,3], strides=1, padding='same', activation=tf.nn.relu)
        G_conv2 = tf.layers.conv2d(G_conv1, filters=1, kernel_size=[3,3], strides=1, padding='same')
        G_logit = tf.reshape(G_conv2, [-1, 28*28])
        G_prob = tf.nn.sigmoid(G_logit)
        return G_prob

In [11]:
def discriminator(x, reuse=False):
    with tf.variable_scope('D', reuse=reuse) as scope:
        D_reshape1 = tf.reshape(x, [-1, 28, 28, 1])
        D_conv1 = tf.layers.conv2d(D_reshape1, filters=32, kernel_size=[3,3], strides=1, padding='same', activation=tf.nn.relu)
        D_pool1 = tf.layers.max_pooling2d(D_conv1, pool_size=[2,2], strides=2)
        D_conv2 = tf.layers.conv2d(D_conv1, filters=64, kernel_size=[3,3], strides=1, padding='same', activation=tf.nn.relu)
        D_pool2 = tf.layers.max_pooling2d(D_conv2, pool_size=[2,2], strides=2)
        D_reshape2 = tf.reshape(D_pool2, [-1, 7*7*64]) 
        D_fc3 = tf.layers.dense(D_reshape2, units=128, activation=tf.nn.relu)
        D_logit = tf.layers.dense(D_fc3, units=1)
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit

In [12]:
G_sample = generator(Z)
D_prob_real, D_logit_real = discriminator(X)
D_prob_fake, D_logit_fake = discriminator(G_sample, reuse=True)

In [13]:
var = tf.trainable_variables()
var_G = [v for v in var if v.name.startswith("G")]
var_D = [v for v in var if v.name.startswith("D")]

In [14]:
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake

In [15]:
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [16]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=var_D)

In [17]:
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=var_G)

In [ ]:
saver = tf.train.Saver(max_to_keep=5)

samples_dict = {}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        if i % 1 == 0:
            n_samples = 16
            samples = sess.run(G_sample, feed_dict={Z: sample_Z(n_samples, Z_dim)})
            samples_dict[i] = samples
        
        X_mb, _ = mnist.train.next_batch(batch_size)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})
        
        if i % 1 == 0:
            print "Iter: {}".format(i)
            print "D loss: {:.4}".format(D_loss_curr)
            print "G loss: {:.4}".format(G_loss_curr)
            
            saver.save(sess, 'save/GAN_conv/GAN_conv', global_step=i)

In [127]:
G_dense = tf.layers.dense(Z, units=14*14*200, activation=tf.nn.relu)
G_reshape = tf.reshape(G_dense, [-1, 14, 14, 1])

In [129]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(G_reshape, feed_dict={Z: sample_Z(n_samples, Z_dim)}).shape

(3200, 14, 14, 1)
